<a href="https://colab.research.google.com/github/samnm/notebooks/blob/main/SD_custom_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Dependencies
%cd /content/
!git clone https://github.com/TheLastBen/diffusers | true
!pip install -q typing-extensions==4.1
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q termcolor
!pip install --ignore-installed -q git+https://github.com/TheLastBen/diffusers
!pip install -q accelerate==0.12.0
!pip install -q OmegaConf
!pip install -q wget
!pip install -q huggingface_hub
!pip install -U -q --no-cache-dir gdown
!wget https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/Deps
!mv Deps Deps.7z
!7z x Deps.7z
!cp -r /content/usr/local/lib/python3.7/dist-packages /usr/local/lib/python3.7/
!rm Deps.7z
!rm -rf /content/usr
!sed -i 's@else prefix + ": "@else prefix + ""@g' /usr/local/lib/python3.7/dist-packages/tqdm/std.py

In [ ]:
#@markdown # xformers

from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import wget
import time

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    raise Exception('[1;31mit seems that your GPU is not supported at the moment')

if (gpu=='T4'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %cd /usr/local/lib/python3.7/diffusers/models/
  !rm /usr/local/lib/python3.7/diffusers/models/attention.py
  wget.download('https://raw.githubusercontent.com/huggingface/diffusers/269109dbfbbdbe2800535239b881e96e1828a0ef/src/diffusers/models/attention.py') 
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl

clear_output()
print('[1;32mDONE !')

In [ ]:
import subprocess
weights_url = "" #@param {type:"string"}
ckpt_path = '/content/weights.ckpt'
if weights_url.startswith('gs://'):
    subprocess.check_output(['gcloud', 'storage', 'cp', weights_url, ckpt_path])
else:
    subprocess.check_output(['wget', weights_url, '-O', ckpt_path])

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/039958eae55ff0700cfb42a7e72739575ab341f1/scripts/convert_original_stable_diffusion_to_diffusers.py
!python /content/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "$ckpt_path" --dump_path /content/stable-diffusion-v1-5
!rm /content/convert_original_stable_diffusion_to_diffusers.py

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("/content/stable-diffusion-v1-5")
pipe = pipe.to("cuda")

In [ ]:
prompt = "" #@param {type:"string"}
image = pipe(prompt).images[0]
image.save("out.png")